In [27]:
import spiketoolkit as st
import spikeextractors as se
import numpy as np

In [6]:
workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")

Could not load plane information. Assuming probe is in yz plane


In [8]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [12]:
sorting = st.sorters.run_mountainsort4(recording)

In [49]:
spike_times = []
spike_clusters = []
for unit_id in sorting.get_unit_ids():
    spike_times_unit = recording.frame_to_time(sorting.get_unit_spike_train(unit_id))
    spike_clusters_unit = np.zeros(len(spike_times_unit)) + unit_id
    spike_times.append(spike_times_unit)
    spike_clusters.append(spike_clusters_unit)
spike_times = np.concatenate(np.asarray(spike_times), 0)
spike_clusters = np.concatenate(np.asarray(spike_clusters), 0)
sort_indices = np.argsort(spike_times)


spike_times = spike_times[sort_indices]
spike_clusters = spike_clusters[sort_indices]

In [51]:
spike_times[0]

0.003125

In [52]:
print(recording.frame_to_time(sorting.get_unit_spike_train(39)[0]))

0.003125


In [50]:
spike_clusters

array([39., 20., 34., ..., 34., 49., 53.])

In [47]:
spike_times.shape

(10501,)

In [48]:
spike_times[:1000]

array([0.003125  , 0.00628125, 0.00678125, 0.009125  , 0.0144375 ,
       0.0149375 , 0.01634375, 0.02021875, 0.02071875, 0.02321875,
       0.02425   , 0.02828125, 0.02878125, 0.0388125 , 0.0418125 ,
       0.04228125, 0.04396875, 0.05309375, 0.0533125 , 0.05359375,
       0.05515625, 0.0623125 , 0.0678125 , 0.07434375, 0.07478125,
       0.0800625 , 0.0856875 , 0.08571875, 0.0879375 , 0.090625  ,
       0.09440625, 0.1019375 , 0.10421875, 0.1080625 , 0.109125  ,
       0.11415625, 0.11471875, 0.1155    , 0.1214375 , 0.12746875,
       0.1289375 , 0.13128125, 0.1365625 , 0.137375  , 0.14790625,
       0.14940625, 0.1499375 , 0.15125   , 0.1516875 , 0.15275   ,
       0.1556875 , 0.1605625 , 0.1626875 , 0.16728125, 0.16821875,
       0.1715    , 0.17425   , 0.17928125, 0.18259375, 0.184875  ,
       0.19171875, 0.198     , 0.1986875 , 0.19909375, 0.20046875,
       0.20325   , 0.2090625 , 0.2131875 , 0.21328125, 0.22015625,
       0.220625  , 0.22165625, 0.22703125, 0.23003125, 0.23153